# Scrape Sports Reference for CBB data back to 2002

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)
    
import os.path
    

In [2]:
yearList=[str(yr) for yr in np.arange(2017,2001,-1)]

In [3]:
fileIsLocal=False
for year in yearList:

    team_data=pd.read_csv('data/combined_team_stats_'+year+'.csv')
    team_names=team_data.School.values    
    fileIsLocal=False
    # get results of all games
    all_games = []
    print('Looping over teams in ',year)
    for team in team_names:
        team = team.lower().replace(" ", "-")
        sched_url = team_data.loc[team_data.School==team,'sched_url'].values[0]
        localFile='data/schedule/'+sched_url.split('schools/')[-1].replace('/','-')
        urlRoot = "https://www.sports-reference.com/"
        url = urlRoot + sched_url
        
        fileIsLocal = os.path.isfile(localFile)
        
        if fileIsLocal:
            with open(localFile, 'r') as myfile:
                data=myfile.read()        
        else:
            r = requests.get(url)        
            open(localFile, 'wb').write(r.content)
            data = r.text
        soup = BeautifulSoup(data, "html5lib")
        
        hasImgFile=os.path.isfile('data/logos/'+team+'.png')
        if not hasImgFile:
            teamImgSrc=soup.find("img", {"class": "teamlogo"})['src']
            urllib.request.urlretrieve(teamImgSrc,'data/logos/'+team+'.png')

        # opponent names
        opp_names = soup.findAll("td", {"data-stat": "opp_name"})
        opp_names = [node.getText().lower().replace(" ", "-") for node in opp_names]

        # result of game
        game_result = soup.findAll("td", {"data-stat": "game_result"})
        game_result = [node.getText() for node in game_result]
        game_result = [0 if node == 'L' else 1 for node in game_result]

        for opp, result in zip(opp_names, game_result):
            if opp in team_names:
                all_games.append((team, opp, result))

    combinedDF=[]
    print('Looping over games in ',year)
    for game in all_games:
        team_one = team_data[team_data['School'] == game[0]]
        column1 = team_one.columns[1:]    
        team_one[column1].values[0]

        team_two = team_data[team_data['School'] == game[1]]
        column2 = team_two.columns[1:]    
        team_two[column2].values[0]

        combinedDF.append(list(team_one[column1].values[0])+list(team_two[column2].values[0])+[game[2]])
    combinedColumns=[c+'_1' for c in column1]+[c+'_2' for c in column2]+['outcome']

    game_data=pd.DataFrame(combinedDF,columns=combinedColumns)
    game_data.to_csv("data/games/all_games_" + year + ".csv")




Looping over teams in  2017
Looping over games in  2017
Looping over teams in  2016
Looping over games in  2016
Looping over teams in  2015
Looping over games in  2015
Looping over teams in  2014
Looping over games in  2014
Looping over teams in  2013
Looping over games in  2013
Looping over teams in  2012
Looping over games in  2012
Looping over teams in  2011
Looping over games in  2011
Looping over teams in  2010
Looping over games in  2010
Looping over teams in  2009
Looping over games in  2009
Looping over teams in  2008
Looping over games in  2008
Looping over teams in  2007
Looping over games in  2007
Looping over teams in  2006
Looping over games in  2006
Looping over teams in  2005
Looping over games in  2005
Looping over teams in  2004
Looping over games in  2004
Looping over teams in  2003
Looping over games in  2003
Looping over teams in  2002
Looping over games in  2002


In [4]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

dbname = 'ncaabb'
username = 'gshau' # change this to your username
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))



postgres://gshau@localhost/ncaabb
True


In [5]:
game_data.columns=[gk.replace('%','') for gk in game_data.keys()]

In [6]:
game_data.to_sql('game_data', engine, if_exists='replace')

In [31]:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM game_data where outcome > 0.5 and wpct_1 > 0.5 LIMIT 5;
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)
birth_data_from_sql

,index,School_1,wpct_1,SRS_1,SOS_1,TmPts_1,OppPts_1,Pace_1,ORtg_1,FTr_1,...,AdjEM_2,AdjO_2,AdjD_2,AdjT_2,Luck_2,AdjEM.1_2,OppO_2,OppD_2,NCSOS_AdjEM_2,outcome
0,53,alabama-a&m,0.655,-11.33,-15.81,2318,2123,None,None,0.238,...,5.01,104.0,99.0,63.3,0.051,2.37,105.2,102.9,3.03,1
1,54,alabama-a&m,0.655,-11.33,-15.81,2318,2123,None,None,0.238,...,18.25,111.4,93.1,67.9,0.107,8.93,106.9,98.0,-0.96,1
2,56,alabama-a&m,0.655,-11.33,-15.81,2318,2123,None,None,0.238,...,-18.91,97.3,116.2,74.0,0.006,-10.16,98.1,108.3,13.81,1
3,58,alabama-a&m,0.655,-11.33,-15.81,2318,2123,None,None,0.238,...,-13.67,101.1,114.8,69.4,-0.041,-12.87,96.6,109.5,2.61,1
4,59,alabama-a&m,0.655,-11.33,-15.81,2318,2123,None,None,0.238,...,-12.62,90.1,102.7,63.1,0.081,-14.95,95.8,110.8,-5.12,1
